In [1]:
import cudf

data=cudf.read_csv("/home/pavit21178/Nalin_OFF/Data/OFF_FNDDS_COLS_NOVA.csv",index_col=0)
# keep only data w 1,2,3,4 in the novaclass column
data = data[data['novaclass'].isin([1,2,3,4])]







In [2]:
# use random forest to predict the NOVA group
from cuml.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from cuml.metrics import accuracy_score

X = data.drop('novaclass', axis=1)
y=data['novaclass']



In [3]:
from cuml.preprocessing import SimpleImputer
import cupy as cp
imp_mean = SimpleImputer(missing_values=cp.nan, strategy='mean')


In [4]:
from cuml.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train=imp_mean.fit_transform(X_train)
X_test=imp_mean.transform(X_test)




In [7]:
from catboost import CatBoostClassifier, Pool
X_train_np=X_train.to_pandas()
X_test_np=X_test.to_pandas()
y_train_np=y_train.to_pandas()
y_test_np=y_test.to_pandas()
train_data = Pool(data=X_train_np, label=y_train_np)
model = CatBoostClassifier(
    task_type="GPU"
)
model.fit(train_data)



Learning rate set to 0.246997
0:	learn: 1.1207058	total: 14.9ms	remaining: 14.9s
1:	learn: 0.9910828	total: 26.3ms	remaining: 13.1s
2:	learn: 0.9147642	total: 40.6ms	remaining: 13.5s
3:	learn: 0.8587649	total: 49.8ms	remaining: 12.4s
4:	learn: 0.8206927	total: 58.4ms	remaining: 11.6s
5:	learn: 0.7900750	total: 67.6ms	remaining: 11.2s
6:	learn: 0.7710914	total: 76ms	remaining: 10.8s
7:	learn: 0.7534572	total: 84.8ms	remaining: 10.5s
8:	learn: 0.7366717	total: 93.6ms	remaining: 10.3s
9:	learn: 0.7248515	total: 102ms	remaining: 10.1s
10:	learn: 0.7166396	total: 111ms	remaining: 9.94s
11:	learn: 0.7092322	total: 119ms	remaining: 9.83s
12:	learn: 0.7017990	total: 129ms	remaining: 9.78s
13:	learn: 0.6952041	total: 137ms	remaining: 9.65s
14:	learn: 0.6907977	total: 147ms	remaining: 9.67s
15:	learn: 0.6857405	total: 177ms	remaining: 10.9s
16:	learn: 0.6817459	total: 203ms	remaining: 11.8s
17:	learn: 0.6777835	total: 233ms	remaining: 12.7s
18:	learn: 0.6749589	total: 241ms	remaining: 12.4s
19:	

In [8]:
y_pred = model.predict(X_test_np)

In [9]:
print(accuracy_score(y_test,y_pred))
y_test_np=y_test.to_numpy()
# y_pred_np=y_pred.to_numpy()
from sklearn.metrics import classification_report
print(classification_report(y_test_np, y_pred))

0.7775048017501831
              precision    recall  f1-score   support

         1.0       0.78      0.64      0.70     20118
         2.0       0.88      0.57      0.70     11358
         3.0       0.71      0.46      0.56     34779
         4.0       0.78      0.93      0.85    108760

    accuracy                           0.78    175015
   macro avg       0.79      0.65      0.70    175015
weighted avg       0.78      0.78      0.76    175015



In [19]:
param_grid_border_count = {
    'iterations': [100, 500, 300],
    'depth': [4, 6, 8, 10, 12,-1],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'border_count': [32, 64, 128],
    'bagging_temperature': [0, 0.5, 1, 2],
    'random_strength': [1, 2, 5, 10],
    'rsm': [0.6, 0.8, 1.0],
    'scale_pos_weight': [1, 2, 5, 10],
    'min_data_in_leaf': [1, 10, 20, 50],
    'od_type': ['IncToDec', 'Iter'],
    'od_wait': [20, 50, 100]
}


In [20]:
from sklearn.model_selection import RandomizedSearchCV

# Set up the RandomizedSearchCV
model = CatBoostClassifier(task_type="GPU", logging_level='Silent',devices='0')

grid_search_border_count = RandomizedSearchCV(estimator=model, param_distributions=param_grid_border_count, cv=5, n_jobs=-1, verbose=1)

# Fit the model
grid_search_border_count.fit(X_train_np, y_train_np)

# Best parameters found
print("Best parameters found (border_count): ", grid_search_border_count.best_params_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


uncaught exception:
    address -> 0x416a3c50410
    what() -> "/src/catboost/catboost/cuda/cuda_lib/cuda_base.cpp:16: CUDA error 2: out of memory"
    type -> TCatBoostException


KeyboardInterrupt: 

In [18]:
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score: ", random_search.best_score_)

# Evaluate the best model on the imputed test set
best_model = random_search.best_estimator_
test_score = best_model.score(X_test_np, y_test_np)
print("Test set score of the best model: ", test_score)

Best parameters found:  {'subsample': 0.8, 'scale_pos_weight': 1, 'reg_lambda': 0.1, 'reg_alpha': 1, 'num_leaves': 127, 'n_estimators': 1000, 'min_split_gain': 0, 'min_child_weight': 1, 'max_depth': -1, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'boosting_type': 'gbdt'}
Best cross-validation score:  0.8057180813073165
Test set score of the best model:  0.8078507556495158


In [21]:
lgbm=CatBoostClassifier(task_type="GPU", logging_level='Silent',devices='0')

In [22]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    X_train = X_train.to_pandas()
    y_train = y_train.to_pandas()
    X_test = X_test.to_pandas()
    y_test = y_test.to_pandas()

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        train_indices = cp.asnumpy(train_indices)
        val_indices = cp.asnumpy(val_indices)

        X_fold_train, X_fold_val = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_fold_train, y_fold_val = y_train.iloc[train_indices], y_train.iloc[val_indices]

        smote = SMOTE()
        X_fold_train_resampled, y_fold_train_resampled = smote.fit_resample(X_fold_train, y_fold_train)

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)

        y_train_pred = classifier.predict(X_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled, y_train_pred)
        train_scores.append(train_score)

        fold_accuracy_train = accuracy_score(y_fold_train_resampled, y_train_pred)
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled, y_train_pred))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        y_val_pred = classifier.predict(X_fold_val)
        val_score = accuracy_score(y_fold_val, y_val_pred)
        val_scores.append(val_score)

        fold_accuracy_val = accuracy_score(y_fold_val, y_val_pred)
        accuracies_val.append(fold_accuracy_val)

        val_mcc.append(matthews_corrcoef(y_fold_val, y_val_pred))
        val_f1.append(precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test, y_pred)
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))



In [29]:
evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, lgbm, num_folds=10, save_path="/home/pavit21178/Nalin_OFF/Models_44/Catboost", model_name="Catboost_strat_Kfold_Smote")

Average Accuracy Val K-Fold: 71.6841
Average Accuracy Train K-Fold: 81.4997
Average MCC Val K-Fold: 0.5322
Average MCC Train K-Fold: 0.7561
Average F1 Score Val K-Fold: 0.7262
Average F1 Score Train K-Fold: 0.8135
Average Precision Val K-Fold: 0.7523
Average Precision Train K-Fold: 0.8203
Average Recall Val K-Fold: 0.7168
Average Recall Train K-Fold: 0.8150
Accuracy Test: 71.68%
MCC Test: 0.5344
F1 Score Test: 0.7262
Precision Test: 0.7524
Recall Test: 0.7168
Classification Report:
              precision    recall  f1-score   support

         1.0       0.70      0.67      0.69     20118
         2.0       0.41      0.91      0.56     11358
         3.0       0.57      0.58      0.57     34779
         4.0       0.86      0.75      0.80    108760

    accuracy                           0.72    175015
   macro avg       0.63      0.73      0.66    175015
weighted avg       0.75      0.72      0.73    175015

Confusion Matrix Test:
[[13521  3419   994  2184]
 [  191 10352    89   726]
 

In [24]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    X_train = X_train.to_pandas()
    y_train = y_train.to_pandas()
    X_test = X_test.to_pandas()
    y_test = y_test.to_pandas()

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        train_indices = cp.asnumpy(train_indices)
        val_indices = cp.asnumpy(val_indices)

        X_fold_train, X_fold_val = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_fold_train, y_fold_val = y_train.iloc[train_indices], y_train.iloc[val_indices]

        smote = SMOTE()
        X_fold_train_resampled, y_fold_train_resampled = smote.fit_resample(X_fold_train, y_fold_train)

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)

        y_train_pred = classifier.predict(X_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled, y_train_pred)
        train_scores.append(train_score)

        fold_accuracy_train = accuracy_score(y_fold_train_resampled, y_train_pred)
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled, y_train_pred))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        y_val_pred = classifier.predict(X_fold_val)
        val_score = accuracy_score(y_fold_val, y_val_pred)
        val_scores.append(val_score)

        fold_accuracy_val = accuracy_score(y_fold_val, y_val_pred)
        accuracies_val.append(fold_accuracy_val)

        val_mcc.append(matthews_corrcoef(y_fold_val, y_val_pred))
        val_f1.append(precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test, y_pred)
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))



In [30]:
evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, lgbm, num_folds=10, save_path="/home/pavit21178/Nalin_OFF/Models_44/Catboost", model_name="Catboost_Kfold_Smote")

Average Accuracy Val K-Fold: 71.6540
Average Accuracy Train K-Fold: 81.5136
Average MCC Val K-Fold: 0.5323
Average MCC Train K-Fold: 0.7560
Average F1 Score Val K-Fold: 0.7260
Average F1 Score Train K-Fold: 0.8137
Average Precision Val K-Fold: 0.7523
Average Precision Train K-Fold: 0.8197
Average Recall Val K-Fold: 0.7165
Average Recall Train K-Fold: 0.8151
Accuracy Test: 71.73%
MCC Test: 0.5353
F1 Score Test: 0.7267
Precision Test: 0.7528
Recall Test: 0.7173
Classification Report:
              precision    recall  f1-score   support

         1.0       0.70      0.67      0.69     20118
         2.0       0.41      0.91      0.56     11358
         3.0       0.57      0.58      0.57     34779
         4.0       0.86      0.75      0.80    108760

    accuracy                           0.72    175015
   macro avg       0.63      0.73      0.66    175015
weighted avg       0.75      0.72      0.73    175015

Confusion Matrix Test:
[[13560  3429   975  2154]
 [  192 10320    91   755]
 

In [27]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_stratified(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    X_train = X_train.to_pandas()
    y_train = y_train.to_pandas()
    X_test = X_test.to_pandas()
    y_test = y_test.to_pandas()

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        train_indices = cp.asnumpy(train_indices)
        val_indices = cp.asnumpy(val_indices)

        X_fold_train, X_fold_val = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_fold_train, y_fold_val = y_train.iloc[train_indices], y_train.iloc[val_indices]

        X_fold_train_resampled, y_fold_train_resampled = X_fold_train, y_fold_train

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)

        y_train_pred = classifier.predict(X_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled, y_train_pred)
        train_scores.append(train_score)

        fold_accuracy_train = accuracy_score(y_fold_train_resampled, y_train_pred)
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled, y_train_pred))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        y_val_pred = classifier.predict(X_fold_val)
        val_score = accuracy_score(y_fold_val, y_val_pred)
        val_scores.append(val_score)

        fold_accuracy_val = accuracy_score(y_fold_val, y_val_pred)
        accuracies_val.append(fold_accuracy_val)

        val_mcc.append(matthews_corrcoef(y_fold_val, y_val_pred))
        val_f1.append(precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test, y_pred)
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))



In [31]:
evaluate_classifier_with_stratified(X_train, y_train, X_test, y_test, lgbm, num_folds=10, save_path="/home/pavit21178/Nalin_OFF/Models_44/Catboost", model_name="Catboost_strat")

Average Accuracy Val K-Fold: 77.6608
Average Accuracy Train K-Fold: 78.4583
Average MCC Val K-Fold: 0.5693
Average MCC Train K-Fold: 0.5857
Average F1 Score Val K-Fold: 0.7628
Average F1 Score Train K-Fold: 0.7714
Average Precision Val K-Fold: 0.7733
Average Precision Train K-Fold: 0.7827
Average Recall Val K-Fold: 0.7766
Average Recall Train K-Fold: 0.7846
Accuracy Test: 77.70%
MCC Test: 0.5727
F1 Score Test: 0.7635
Precision Test: 0.7747
Recall Test: 0.7770
Classification Report:
              precision    recall  f1-score   support

         1.0       0.78      0.64      0.70     20118
         2.0       0.89      0.58      0.70     11358
         3.0       0.71      0.46      0.56     34779
         4.0       0.78      0.93      0.85    108760

    accuracy                           0.78    175015
   macro avg       0.79      0.65      0.70    175015
weighted avg       0.77      0.78      0.76    175015

Confusion Matrix Test:
[[ 12832    117    809   6360]
 [   136   6532     75  